In [1]:
import csv
import numpy as np

def parseCSVData(file,data,maxCount):
    rowIndex = 0
    rowCount = 0
    with open(file, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            if rowCount == maxCount:
                break
            rowCount+=1
            if rowIndex !=0:
                element = {}
                fieldnames = ("PostId","PostCreationDate","OwnerUserId","OwnerCreationDate","ReputationAtPostCreation","OwnerUndeletedAnswerCountAtPostTime","Title","BodyMarkdown","Tag1","Tag2","Tag3","Tag4","Tag5","PostClosedDate","OpenStatus")
                index = 0
                for i in row:
                    element[fieldnames[index]] = i
                    if index == 0 or index == 5 or index == 4:   
                        element[fieldnames[index]] = int(i)
                    index +=1
                data.append(element)
            else:
                rowIndex = 1
    return


In [2]:
from collections import defaultdict

def dataCount(ownerCount,tagCount,openStatusCount,dateClosed,data):
    tag1NotUsed = 0
    tag2NotUsed = 0
    tag3NotUsed = 0
    tag4NotUsed = 0
    tag5NotUsed = 0
    for i in data:
        ownerCount[i['OwnerUserId']]+=1
        openStatusCount[i['OpenStatus']]+=1
        if i['Tag1'] != '':
            tagCount[i['Tag1']]+=1
        else:
            tag1NotUsed+=1
            
        if i['Tag2'] != '':
            tagCount[i['Tag2']]+=1
        else:
            tag2NotUsed+=1
            
        if i['Tag3'] != '':
            tagCount[i['Tag3']]+=1
        else:
            tag3NotUsed+=1
            
        if i['Tag4'] != '':
            tagCount[i['Tag4']]+=1
        else:
            tag4NotUsed+=1
        if i['Tag5'] != '':
            tagCount[i['Tag5']]+=1
        else:
            tag5NotUsed+=1

        if i['PostClosedDate']!='':
            dateClosed+=1
        
    dateClosed = dateClosed
    return (tag1NotUsed,tag2NotUsed,tag3NotUsed,tag4NotUsed,tag5NotUsed)

In [3]:
leData1 = []
max = 100001
parseCSVData('output_Final1.csv',leData1,max)

In [4]:
ownerCount = defaultdict(int)
tagCount = defaultdict(int)
openStatusCount = defaultdict(int)
dateClosed = 0


tagNotUsed = dataCount(ownerCount,tagCount,openStatusCount,dateClosed,leData1)

In [5]:
totalTagNotUsed = 0
for i in tagNotUsed:
    totalTagNotUsed+=i

In [6]:
totalTagNotUsed, (100000)*5 - totalTagNotUsed

(217169, 282831)

In [7]:
tagCount
import operator
sorted_x = sorted(tagCount.items(), key=operator.itemgetter(1))

In [8]:
sorted_x.reverse()
len(sorted_x)

16005

In [9]:
sorted_x
words = [x[0] for x in sorted_x]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
len(wordId)

{'cmdlet': 8690,
 'mdbg': 16004,
 'h.264': 3897,
 'html4': 6573,
 'apache-ivy': 9011,
 'raining': 15564,
 'xgettext': 16002,
 'sqltrace': 14829,
 'dropdownextender': 7527,
 'log-shipping': 16001,
 'legacy-code': 1417,
 'tcp-ip': 9010,
 'hsv': 10426,
 'modbus': 12638,
 'verses': 14025,
 'hanging': 6572,
 'post-grad': 9009,
 'binary-operators': 9008,
 'database-installation': 9007,
 'self-study': 15999,
 'inline-editing': 6104,
 'typeerror': 6568,
 'jml': 15571,
 'windows-ce': 739,
 'ibm-datapower': 15996,
 'errortext': 15995,
 'coreanimation': 6570,
 'diplomatic-terminology': 15994,
 'code-comments': 4455,
 'xunit': 3154,
 'sqlmanagmentstudio': 5274,
 'dependency': 6689,
 'db-schema': 15993,
 'ramdisk': 15992,
 'learningc#': 15991,
 'onsubmit': 9006,
 'barcode-scanner': 15990,
 'winsockets': 4454,
 'filenotfoundexception': 5273,
 'perforce-changelist': 15989,
 'enterprise-architecture': 15988,
 'http-authentication': 3141,
 'regional': 4453,
 'adodbapi': 15987,
 'browser-compatibility':

In [10]:
def feature(datum):
    feat  = [0]*len(words)
    if datum['Tag1']!='':
        feat[wordId[datum['Tag1']]] = 1
    if datum['Tag2']!='':
        feat[wordId[datum['Tag2']]] = 1
    if datum['Tag3']!='':
        feat[wordId[datum['Tag3']]] = 1
    if datum['Tag4']!='':
        feat[wordId[datum['Tag4']]] = 1
    if datum['Tag5']!='':
        feat[wordId[datum['Tag5']]] = 1
    
    feat.append(datum['ReputationAtPostCreation'])
    feat.append(datum['OwnerUndeletedAnswerCountAtPostTime'])
    
    return feat

In [11]:
def result(datum):
    if datum['OpenStatus']!='open':
        return 1
    
    return 0

In [12]:
X = [feature(d) for d in leData1]


In [13]:
y = [result(d) for d in leData1]

len(leData1), len(X), len(y)

(100000, 100000, 100000)

In [14]:
leData1 = ''
len(leData1)
len(leData1), len(X), len(y)

(0, 100000, 100000)

In [15]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()

In [16]:
clf.fit(X,y)
print 'done'

done


In [17]:
predictions = clf.predict(X)
X = 0

In [45]:
def classificationVals(predictions,y):
    index = 0
    truePositive = 0
    trueNegative = 0
    falsePositive = 0
    falseNegative = 0
    for x in predictions:
        if y[index] == x:
            if x == 1:
                truePositive+=1
            else:
                trueNegative+=1
        else:
            if y[index] == 1 and x == 0:
                falseNegative+=1
            else:
                falsePositive +=1
        index+=1
    return truePositive,trueNegative,falsePositive,falseNegative
# 96849

In [46]:
clasVal = classificationVals(predictions,y)
clasVal

(386, 96463, 158, 2993)

In [40]:
def ClassificationAccuracy(TP,TN,FP,FN):
    return (1.0*(TP+TN)/(TN + TP + FP +FN))

In [83]:
def ErrorRate(TP,TN,FP,FN):
    return (1.0*(FP+FN)/(TN + TP + FP +FN))

In [47]:
ClassificationAccuracy(clasVal[0],clasVal[1],clasVal[2],clasVal[3])
# 0.96849

0.96849

In [78]:
coefficients = clf.coef_[0]

In [22]:
len(predictions)-countRight

3151

In [23]:
len(clf.coef_[0])

16007

## Writing the Model Info

In [28]:
f = open('Model1.txt', 'w')
f.write('Model one using all the tags from the first 100,000 entries, 16006 feature representation\n')
for x in clf.coef_[0]:
    f.write(str(x))
    f.write('\n')
f.close()

In [25]:
len(wordId)

16005

In [32]:
f = open('Model1_Tag_Dictionary.txt','w')

for x in wordId:
    f.write(x + "," +str(wordId[x]))
    f.write('\n')
f.close

<function close>

## Check performance against unseen data, with 50k samples

In [70]:
testData = []
max = 50001
parseCSVData('output_Final3.csv',testData,max)

In [71]:
len(testData)

50000

In [72]:
y = 0

In [73]:
def featureTest(datum):
    feat  = [0]*len(words)
    if datum['Tag1']!='':
        if datum['Tag1'] in wordId:
            feat[wordId[datum['Tag1']]] = 1
    if datum['Tag2']!='':
         if datum['Tag2'] in wordId:
            feat[wordId[datum['Tag2']]] = 1
    if datum['Tag3']!='':
         if datum['Tag3'] in wordId:
            feat[wordId[datum['Tag3']]] = 1
    if datum['Tag4']!='':
         if datum['Tag4'] in wordId:
            feat[wordId[datum['Tag4']]] = 1
    if datum['Tag5']!='':
         if datum['Tag5'] in wordId:
            feat[wordId[datum['Tag5']]] = 1
    
    feat.append(datum['ReputationAtPostCreation'])
    feat.append(datum['OwnerUndeletedAnswerCountAtPostTime'])
    
    return feat

In [74]:
X1 = [featureTest(d) for d in testData]
y = [result(d) for d in testData]


In [75]:
Testpredictions = clf.predict(X1)


In [76]:
clasVal = classificationVals(Testpredictions,y)
clasVal

(16, 48766, 13, 1205)

In [77]:
ClassificationAccuracy(clasVal[0],clasVal[1],clasVal[2],clasVal[3])

0.97564

In [79]:
words[0]

'c#'

In [80]:
index = 0
allFeatures = []
while index < len(words):
    featureword = words[index]
    allFeatures.append((coefficients[index],featureword))
    index+=1

In [81]:
allFeatures.sort()
allFeatures[:20]

[(-1.7541927498124841, 'validation'),
 (-1.6133853094700588, 'tsql'),
 (-1.5130293812435553, 'ms-access'),
 (-1.4914331363527273, 'memory'),
 (-1.4872792927655318, 'wcf'),
 (-1.4527833333306297, 'msbuild'),
 (-1.4287056471397914, 'session'),
 (-1.3818008074795778, 'class'),
 (-1.3784245751732331, 'com'),
 (-1.3756143910474969, 'sorting'),
 (-1.3674756125471301, 'events'),
 (-1.3634261307550097, 'usercontrols'),
 (-1.3410296501564201, 'actionscript-3'),
 (-1.3154371304127321, 'multithreading'),
 (-1.3107528835018387, 'encoding'),
 (-1.3062725045580195, 'asp.net-ajax'),
 (-1.2738136961052531, 'reflection'),
 (-1.2534862833902001, 'pointers'),
 (-1.2452730690081875, 'datetime'),
 (-1.2445550874914511, 'regular-expressions')]

In [82]:
allFeatures.reverse()
allFeatures[:20]

[(3.9629141965120551, 'polls'),
 (3.6011583362067965, 'books'),
 (3.2733806274899968, 'legal'),
 (2.9575760535200808, 'enterprise-development'),
 (2.8501323043584139, 'podcast'),
 (2.6786220574520621, 'outsourcing'),
 (2.665884061600766, 'discussion'),
 (2.569109605124678, 'teamwork'),
 (2.5499409434584801, 'career-development'),
 (2.4798313767856413, 'ebook'),
 (2.4700262561460646, 'book'),
 (2.2164755906328151, 'jobs'),
 (2.1686838003455979, 'motivation'),
 (2.1587624037958926, 'patents'),
 (2.053354949101601, 'ergonomics'),
 (1.9765912356052995, 'study'),
 (1.9690624817005262, 'business'),
 (1.9502891766949073, 'tutorials'),
 (1.9464799305098792, '.net-3.0'),
 (1.9334925149527047, 'copyright')]

In [91]:
closeToNoVal = 0
newFeautures = []
newCoefficients = []
for i in allFeatures:
    if i[0] < 0.02  and i[0]>(-0.02):
        closeToNoVal+=1
    else:
        newFeautures.append(i)

In [92]:
closeToNoVal, len (allFeatures)

(4471, 16005)

In [94]:
len(newFeautures), newFeautures[0]
newFeautures.reverse

<function reverse>

In [95]:
wordId

{'cmdlet': 8690,
 'mdbg': 16004,
 'h.264': 3897,
 'html4': 6573,
 'apache-ivy': 9011,
 'raining': 15564,
 'xgettext': 16002,
 'sqltrace': 14829,
 'dropdownextender': 7527,
 'log-shipping': 16001,
 'legacy-code': 1417,
 'tcp-ip': 9010,
 'hsv': 10426,
 'modbus': 12638,
 'verses': 14025,
 'hanging': 6572,
 'post-grad': 9009,
 'binary-operators': 9008,
 'database-installation': 9007,
 'self-study': 15999,
 'inline-editing': 6104,
 'typeerror': 6568,
 'jml': 15571,
 'windows-ce': 739,
 'ibm-datapower': 15996,
 'errortext': 15995,
 'coreanimation': 6570,
 'diplomatic-terminology': 15994,
 'code-comments': 4455,
 'xunit': 3154,
 'sqlmanagmentstudio': 5274,
 'dependency': 6689,
 'db-schema': 15993,
 'ramdisk': 15992,
 'learningc#': 15991,
 'onsubmit': 9006,
 'barcode-scanner': 15990,
 'winsockets': 4454,
 'filenotfoundexception': 5273,
 'perforce-changelist': 15989,
 'enterprise-architecture': 15988,
 'http-authentication': 3141,
 'regional': 4453,
 'adodbapi': 15987,
 'browser-compatibility':

In [100]:

newWords = [x[1] for x in newFeautures]
newCoefficients = [x[0] for x in newFeautures]
newWordId = dict(zip(newWords, range(len(newWords))))
newWordSet = set(words)
len(newWordId)

11534

In [103]:
newWordId['raining'],newCoefficients[4887],newFeautures[4887]

(4887, -0.035319932431779659, (-0.035319932431779659, 'raining'))

## Test with fewer features

In [99]:
def featureTest_Less(datum):
    feat  = [0]*len(newWords)
    if datum['Tag1']!='':
        if datum['Tag1'] in newWordId:
            feat[newWordId[datum['Tag1']]] = 1
    if datum['Tag2']!='':
         if datum['Tag2'] in newWordId:
            feat[newWordId[datum['Tag2']]] = 1
    if datum['Tag3']!='':
         if datum['Tag3'] in newWordId:
            feat[newWordId[datum['Tag3']]] = 1
    if datum['Tag4']!='':
         if datum['Tag4'] in newWordId:
            feat[newWordId[datum['Tag4']]] = 1
    if datum['Tag5']!='':
         if datum['Tag5'] in newWordId:
            feat[newWordId[datum['Tag5']]] = 1
    
    feat.append(datum['ReputationAtPostCreation'])
    feat.append(datum['OwnerUndeletedAnswerCountAtPostTime'])
    
    return feat